In [2]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import statistics as stats
import scipy
from scipy.stats.stats import pearsonr
from matplotlib import style
import warnings
import seaborn as sns
import requests
import bs4 as bs
import os
import csv
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
warnings.filterwarnings('ignore') 

In [3]:
data = pd.read_excel('stock_returns_data.xlsx')

In [38]:
df = data[['Names Date','Ticker Symbol','Returns']]

In [39]:
df = df[pd.to_numeric(df['Returns'], errors='coerce').notnull()]

In [40]:
df['Date'] = df['Names Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
df
#date = dt(year=int(dates[0:4]), month=int(dates[4:6]), day=int(dates[6:8]))
#int2date(dates)

,Names Date,Ticker Symbol,Returns,Date
0,20070131,EWST,0.023279,2007-01-31
1,20070228,EWST,0.261621,2007-02-28
2,20070330,EWST,0.019732,2007-03-30
3,20070430,EWST,0.002764,2007-04-30
4,20070531,EWST,0.043418,2007-05-31
...,...,...,...,...
1048568,20191129,VT,0.025614,2019-11-29
1048569,20191231,VT,0.034494,2019-12-31
1048572,20080731,SZR,0.076445,2008-07-31
1048573,20080829,SZR,-0.040735,2008-08-29


In [228]:
df['Returns']=df['Returns']*100
df1 = df[['Date','Ticker Symbol','Returns']]
df1

,Date,Ticker Symbol,Returns
0,2007-01-31,EWST,2.3279
1,2007-02-28,EWST,26.1621
2,2007-03-30,EWST,1.9732
3,2007-04-30,EWST,0.2764
4,2007-05-31,EWST,4.3418
...,...,...,...
1048568,2019-11-29,VT,2.5614
1048569,2019-12-31,VT,3.4494
1048572,2008-07-31,SZR,7.6445
1048573,2008-08-29,SZR,-4.0735


In [229]:
df1 = df1[(df1['Date'] > '2015-01-01') & (df1['Date'] < '2020-01-31')]
df1

,Date,Ticker Symbol,Returns
96,2015-01-30,EGAS,-11.5245
97,2015-02-27,EGAS,3.5897
98,2015-03-31,EGAS,-0.0495
99,2015-04-30,EGAS,1.1044
100,2015-05-29,EGAS,-0.6951
...,...,...,...
1048565,2019-08-30,VT,-2.1022
1048566,2019-09-30,VT,2.2762
1048567,2019-10-31,VT,2.7934
1048568,2019-11-29,VT,2.5614


In [230]:
df1 = df1.dropna(subset=['Ticker Symbol'])

In [231]:
#start = 2015,01,01
#end = 2019,12,31
end = dt.date(2019,12,31)
start = dt.date(2014,12,31)
ticker = '^GSPC'
df_market = web.DataReader(ticker,'yahoo',start,end)
df_market = df_market.reset_index()
df_market[[f'{ticker} Adj Close']] = df_market[['Adj Close']]
df_market=df_market[['Date',f'{ticker} Adj Close']].resample('M', on='Date').last()
df_market[f'{ticker} Monthly Return'] = df_market[f'{ticker} Adj Close'].pct_change()*100
df_market.dropna(inplace=True)
df_market = df_market[['Date',f'{ticker} Monthly Return']]
#df_market
return_market = df_market[f'{ticker} Monthly Return'].sum()
print(return_market)

48.76399191842387


In [232]:
df_market['Ticker Symbol']='SP500'
df_market['Returns']=df_market['^GSPC Monthly Return']
#df_market = df_market.reset_index()
df_market1 = df_market[['Date','Ticker Symbol','Returns']]
dfAll = pd.concat([df1,df_market1])
dfAll = dfAll.reset_index()
dfAll = dfAll[['Date','Ticker Symbol', 'Returns']]
dfAll = dfAll[(dfAll['Date'] > '2015-01-01') & (dfAll['Date'] < '2019-12-31')]
dfAll

,Date,Ticker Symbol,Returns
0,2015-01-30,EGAS,-11.5245
1,2015-02-27,EGAS,3.5897
2,2015-03-31,EGAS,-0.0495
3,2015-04-30,EGAS,1.1044
4,2015-05-29,EGAS,-0.6951
...,...,...,...
412683,2019-07-31,SP500,1.31282
412684,2019-08-30,SP500,-1.80916
412685,2019-09-30,SP500,1.71812
412686,2019-10-31,SP500,2.04318


In [233]:
#finding stocks in the data (excel sheet) that have data from 2015-2019 (returns) and 2015-2017 (beta)
stock_count = dfAll.groupby('Ticker Symbol')['Returns'].count()
stock_count = stock_count[stock_count == 59]
stocks = [*stock_count.keys()]
stocks

['A',
 'AADR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABDC',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ACAD',
 'ACC',
 'ACCO',
 'ACGL',
 'ACH',
 'ACHC',
 'ACHN',
 'ACIW',
 'ACLS',
 'ACM',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRX',
 'ACST',
 'ACTG',
 'ACU',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADBE',
 'ADC',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRU',
 'ADS',
 'ADSK',
 'ADTN',
 'ADX',
 'ADXS',
 'AE',
 'AEE',
 'AEG',
 'AEGN',
 'AEHR',
 'AEIS',
 'AEL',
 'AEM',
 'AEO',
 'AEP',
 'AER',
 'AERI',
 'AES',
 'AEY',
 'AEZS',
 'AFB',
 'AFG',
 'AFH',
 'AFL',
 'AFMD',
 'AFT',
 'AG',
 'AGCO',
 'AGD',
 'AGEN',
 'AGG',
 'AGI',
 'AGIO',
 'AGNC',
 'AGND',
 'AGO',
 'AGRO',
 'AGRX',
 'AGTC',
 'AGX',
 'AGYS',
 'AGZD',
 'AHC',
 'AHH',
 'AHPI',
 'AHT',
 'AI',
 'AIA',
 'AIF',
 'AIG',
 'AIMC',
 'AIN',
 'AINC',
 'AINV',
 'AIR',
 'AIRI',
 'AIRR',


In [234]:
#Dataframe with all the stocks that have all the required data (date[2015,2019],monthly returns)
dfAll1 = dfAll.loc[dfAll['Ticker Symbol'].isin(stocks)]
dfAll1

,Date,Ticker Symbol,Returns
55,2015-01-30,JJSF,-9.7913
56,2015-02-27,JJSF,3.1288
57,2015-03-31,JJSF,5.801
58,2015-04-30,JJSF,-2.2212
59,2015-05-29,JJSF,3.326
...,...,...,...
412683,2019-07-31,SP500,1.31282
412684,2019-08-30,SP500,-1.80916
412685,2019-09-30,SP500,1.71812
412686,2019-10-31,SP500,2.04318


In [235]:
###Beta calculations###
###Want monthly return beta from 2015 to 2017###
dfBeta = dfAll1[(dfAll1['Date'] > '2015-01-01') & (dfAll1['Date'] < '2017-12-31')]
dfBeta = dfBeta.reset_index()
dfBeta = dfBeta[['Date','Ticker Symbol','Returns']]
dfBeta

,Date,Ticker Symbol,Returns
0,2015-01-30,JJSF,-9.7913
1,2015-02-27,JJSF,3.1288
2,2015-03-31,JJSF,5.801
3,2015-04-30,JJSF,-2.2212
4,2015-05-29,JJSF,3.326
...,...,...,...
160339,2017-08-31,SP500,0.0546433
160340,2017-09-29,SP500,1.9303
160341,2017-10-31,SP500,2.21881
160342,2017-11-30,SP500,2.80826


In [236]:
stdev_stock = dfBeta.groupby('Ticker Symbol')['Returns'].std()
stdev_stock

Ticker Symbol
A        6.151584
AADR     4.083586
AAL      9.241575
AAMC    30.317984
AAME     8.044327
          ...    
ZNGA     9.399612
ZNH     15.458949
ZTR      3.265853
ZTS      5.680491
ZUMZ    14.955430
Name: Returns, Length: 4454, dtype: float64

In [237]:
stdev_market = stdev_stock['SP500']
stdev_market

2.897270113388364

In [238]:
market_quotient = (stdev_stock / stdev_market)
market_quotient['AAPL']

2.298367142215951

In [275]:
###table to find correlations
symbols = dfBeta.groupby('Ticker Symbol')
market_df = symbols.get_group('SP500')
market_df = market_df[['Date','Returns']].set_index('Date')
table = dfBeta.pivot(index='Date',columns='Ticker Symbol',values='Returns')  
table
#table.to_excel('betaReturnsNYSE.xlsx')


Ticker Symbol,A,AADR,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,...,ZION,ZIOP,ZIXI,ZMLP,ZN,ZNGA,ZNH,ZTR,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-30,-7.4988,-2.5773,-8.4841,-48.4071,-0.9926,3.5656,-20.3209,-2.5904,-0.1758,6.1424,...,-15.9593,76.5286,-1.6667,0.4197,9.4891,-3.7594,10.8787,-1.2134,-0.5043,-3.4688
2015-02-27,11.7554,7.5791,-2.2005,10.6563,0.2506,-5.8433,39.5973,3.2095,-2.5598,10.0461,...,11.7488,25.8101,12.9944,2.706,30.6667,-10.1563,-5.9245,2.3127,7.8633,4.103
2015-03-31,-1.3267,-1.2917,10.1879,4.5637,0.75,-4.9547,11.218,8.9738,-3.3434,-3.1372,...,0.9912,-4.3517,-1.75,-4.6253,-11.2245,23.913,44.5648,-1.495,0.4339,3.6837
2015-04-30,-0.4332,4.0653,-8.327,20.9961,-1.2407,20.0989,2.8818,-2.2829,-4.4692,0.5786,...,4.9444,-18.1987,8.397,6.4739,16.092,-14.0351,36.071,-0.0804,-3.8605,-21.2174
2015-05-29,-0.4351,0.0124,-12.2502,-23.933,-4.0404,3.0294,25.07,-1.2516,7.1469,4.5146,...,2.1352,7.605,7.7465,-4.9292,-1.9802,20.4082,1.407,-0.4492,12.0441,-5.8341
2015-06-30,-6.0937,-3.6773,-5.747,-15.3178,-4.7368,3.4342,-2.7996,-4.3937,4.0008,-3.7266,...,9.8857,26.5823,12.6362,-10.1211,-3.5354,-3.0509,17.3135,-2.9859,-3.1143,-10.8172
2015-07-31,6.1431,3.6491,0.4132,-8.1225,2.7624,2.1265,12.0392,-1.4654,9.3666,-3.2888,...,-1.7173,11.3333,-5.6093,-5.0702,-7.8534,-13.2867,-14.9349,-0.9505,1.5761,-1.9902
2015-08-31,-11.3309,-6.8414,-2.5436,-70.9663,-3.2258,1.8118,5.964,-6.8049,0.597,-6.6117,...,-6.8291,-34.8054,-10.2459,-4.5993,-15.3409,2.0161,-36.6937,-5.7659,-8.2217,-10.6897
2015-09-30,-5.1776,-4.1935,-0.3848,-37.776,12.2222,-4.0292,-8.8792,-6.2863,8.1826,-2.1816,...,-5.0345,3.4443,-3.8813,-13.5602,-4.0268,-9.8814,17.142,-3.4341,-8.2238,-32.9472


In [285]:
table = table.astype(float)
correlation = table.corr()
stock_corr = correlation['SP500']
stock_corr['AAPL']

0.5702997235289048

In [288]:
stock_betas = stock_corr*market_quotient
stock_betas
#stock_betas.to_excel('stock_betas_NYSE.xlsx')

Ticker Symbol
A       1.588297
AADR    0.985100
AAL     1.196599
AAMC    1.935465
AAME    0.461735
          ...   
ZNGA    0.428356
ZNH     1.374188
ZTR     0.765649
ZTS     1.199800
ZUMZ    0.971595
Length: 4454, dtype: float64

In [297]:
###Find the returns for each stock that had data from 2015-2019
###returns will be calculated from 1-1-2018 - 11-29-2019 [OR 1.91667 years]
df_stock_returns = dfAll[(dfAll['Date'] > '2018-01-01') & (dfAll['Date'] < '2019-12-31')]
stock_annual_returns = df_stock_returns.loc[df_stock_returns['Ticker Symbol'].isin(stocks)]
stock_annual_returns

,Date,Ticker Symbol,Returns
91,2018-01-31,JJSF,-8.8191
92,2018-02-28,JJSF,-2.9688
93,2018-03-29,JJSF,1.9951
94,2018-04-30,JJSF,0.6224
95,2018-05-31,JJSF,3.0638
...,...,...,...
412683,2019-07-31,SP500,1.31282
412684,2019-08-30,SP500,-1.80916
412685,2019-09-30,SP500,1.71812
412686,2019-10-31,SP500,2.04318


In [306]:
table1 = stock_annual_returns.pivot(index='Date',columns='Ticker Symbol',values='Returns') 
#table1.dtypes
table1 = table1.astype(float)
total_ret = table1.sum()
annual_returns_stocks = total_ret/1.91667
annual_returns_stocks.to_excel('stock_returns_NYSE.xlsx')

In [309]:
###sort betas into 34 portfolio's with equal weights and find total return
#34 because it is divisible by 4454 (number of tickers)
dataframe = pd.read_excel('stock_betas_NYSE_new.xlsx')

In [323]:
###beta of each 34 equally weighted portfolios of 131 stocks
#
group_size = 131
portfolios = [dataframe.iloc[i:i+group_size] for i in range(0,len(dataframe)-group_size+1,group_size)]   
for portfolio in portfolios:
    print(portfolio['Beta'].mean())

3.626758325876645
2.46455943145955
2.100875745464085
1.8700000666223955
1.6995199266331609
1.586020055512714
1.4818045968095308
1.3924019307479738
1.321453494140014
1.2543099320138758
1.1912406144241245
1.135803035497342
1.0861329607909902
1.0463118332143635
1.0108226936494071
0.9711267301811187
0.933728970455316
0.8925782334802131
0.8469967785302729
0.7939876922017551
0.7475475919084996
0.6934299238969961
0.6366967203196668
0.5846329476776014
0.5239576243213685
0.4490643453601007
0.3715844392113726
0.28330186082753495
0.18695433215849458
0.08780194047705363
0.003675646560095035
-0.06446357027485627
-0.22180785104929948
-1.3203295273830904


In [324]:
###annual returns of each portfolio sorted from highest to lowest beta
group_size = 131
portfolios = [dataframe.iloc[i:i+group_size] for i in range(0,len(dataframe)-group_size+1,group_size)]   
for portfolio in portfolios:
    print(portfolio['Annual Return'].mean())

-7.48602348929204
3.233890028017342
0.6289020592609383
-2.5000771654814646
4.399715282274117
0.06002976616130958
4.490184690153409
1.2350089374554156
0.4393083631012857
4.1406264530758
6.4014758898992135
5.522562449974366
8.200575051107442
5.255261620454402
7.487544709405352
5.2251401992251445
3.775959314295783
5.60160818040927
3.671842270012114
11.118664101626315
2.865070490219262
6.1718732357730595
5.828397430865399
3.936135736690586
3.3407292713503525
4.32625414219326
2.7211798675796537
3.5224642357409235
7.67684944351441
3.5421696113611794
4.393911243247622
3.5966247440047607
10.528295795463002
0.6171940145713131
